<center>
    
# Week 9 Assignment
    
</center>

<div style="text-align: right; width: 15%; margin-left: auto;">
    
Jon Cannaday <br>
DSC 630 <br>
Week 9 <br>
7/31/2024  

</div>


# Movie Recommender System
<br>

<div style="width: 100%; border-bottom: 1px solid black;"></div>

We are creating a customer recomendation engine.

Granularity will movie title.


## Extract and Transform Data

   - ### Libraries


In [1]:
import warnings
import numpy as np
import pandas as pd
warnings.filterwarnings("ignore")

   - ### Data


In [2]:
movies_path = ('C:\\Users\\Owner\\OneDrive\\Desktop\\DataScience\\'
        '2024_Summer_Predictive_Analysis\\movies.csv'
         )
tags_path = ('C:\\Users\\Owner\\OneDrive\\Desktop\\DataScience\\'
        '2024_Summer_Predictive_Analysis\\tags.csv'
       )
ratings_path = ('C:\\Users\\Owner\\OneDrive\\Desktop\\DataScience\\'
        '2024_Summer_Predictive_Analysis\\ratings.csv'
          )
links_path = ('C:\\Users\\Owner\\OneDrive\\Desktop\\DataScience\\'
        '2024_Summer_Predictive_Analysis\\links.csv'
        )



raw_movies = pd.DataFrame(pd.read_csv(movies_path))
raw_tags = pd.DataFrame(pd.read_csv(tags_path))
raw_ratings = pd.DataFrame(pd.read_csv(ratings_path))
raw_links = pd.DataFrame(pd.read_csv(links_path))

   - ### Data Prep


Create a usable movie Dimension. Get genres in 1NF.

In [3]:
raw_movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


Will need to get the genre column in a more usable format. Getting all the genres spanned accross the columns will be the approach.

In [4]:
# Split genres
split_genres = raw_movies['genres'].str.get_dummies(sep='|')

# Boolean
for col in split_genres.columns:
    split_genres[col] = split_genres[col].astype(bool)

In [5]:
clean_movies = pd.concat([raw_movies, split_genres], axis=1)
clean_movies = clean_movies.drop(columns=['(no genres listed)'])
clean_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 22 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   movieId      9742 non-null   int64 
 1   title        9742 non-null   object
 2   genres       9742 non-null   object
 3   Action       9742 non-null   bool  
 4   Adventure    9742 non-null   bool  
 5   Animation    9742 non-null   bool  
 6   Children     9742 non-null   bool  
 7   Comedy       9742 non-null   bool  
 8   Crime        9742 non-null   bool  
 9   Documentary  9742 non-null   bool  
 10  Drama        9742 non-null   bool  
 11  Fantasy      9742 non-null   bool  
 12  Film-Noir    9742 non-null   bool  
 13  Horror       9742 non-null   bool  
 14  IMAX         9742 non-null   bool  
 15  Musical      9742 non-null   bool  
 16  Mystery      9742 non-null   bool  
 17  Romance      9742 non-null   bool  
 18  Sci-Fi       9742 non-null   bool  
 19  Thriller     9742 non-null 

User could create multiple tags for one movie so this will be the lowest level of granularity.

In [6]:
raw_tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


We will drop the timestamp column because it does not provide us with much information.

In [7]:
clean_tags = raw_tags.drop(columns=['timestamp'])

we will drop the timestamp again because it does not prodive us with useful information. 
And force the rating to be a whole nmber to be used as a boolean dummy column

In [8]:
raw_ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [9]:
raw_ratings['rating'] = raw_ratings['rating'].round(0).astype(int)

In [10]:
raw_ratings['rating'] = raw_ratings['rating'].astype('category')

In [11]:
rating_dummies = pd.get_dummies(raw_ratings['rating'], prefix='rating')

for col in rating_dummies.columns:
    rating_dummies[col] = rating_dummies[col].astype(bool)

In [12]:
raw_ratings = pd.concat([raw_ratings, rating_dummies], axis=1)
clean_ratings = raw_ratings.drop(columns=['timestamp'])

In [13]:
clean_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   userId    100836 non-null  int64   
 1   movieId   100836 non-null  int64   
 2   rating    100836 non-null  category
 3   rating_0  100836 non-null  bool    
 4   rating_1  100836 non-null  bool    
 5   rating_2  100836 non-null  bool    
 6   rating_3  100836 non-null  bool    
 7   rating_4  100836 non-null  bool    
 8   rating_5  100836 non-null  bool    
dtypes: bool(6), category(1), int64(2)
memory usage: 2.2 MB


This table will be used if we need to tie into other movie databses and bring in more information. We will ignore this table all together.

In [14]:
raw_links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


   - ### Data Engineering


Plan to stitch together

using raw_tags   
    Left Join raw_ratings useing the movieId and userId	 for both tables 
    retrieve from raw_ratings the rating column	


using raw_tags 
    left join  raw_movies useing the movieId on both tables
    retrieve from raw_movies every column except the movieId



The final table should have the tag rom raw_tag, the rating from raw_rating, and all the columns in raw_movie excluding the movieid


In [15]:
merged_data = pd.merge(clean_tags, clean_ratings, on=['movieId', 'userId'], how='left')
raw_model_data = pd.merge(merged_data, clean_movies, on='movieId', how='left')

In [17]:
# Drop the specified columns
raw_model_data = raw_model_data.drop(columns=['userId', 'movieId', 'tag', 'rating', 'genres'])

In [38]:
clean_model_data = raw_model_data.groupby('title').agg(lambda x: x.mode().iloc[0])

In [39]:
clean_model_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1572 entries, (500) Days of Summer (2009) to eXistenZ (1999)
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   rating_0     1572 non-null   bool 
 1   rating_1     1572 non-null   bool 
 2   rating_2     1572 non-null   bool 
 3   rating_3     1572 non-null   bool 
 4   rating_4     1572 non-null   bool 
 5   rating_5     1572 non-null   bool 
 6   Action       1572 non-null   bool 
 7   Adventure    1572 non-null   bool 
 8   Animation    1572 non-null   bool 
 9   Children     1572 non-null   bool 
 10  Comedy       1572 non-null   bool 
 11  Crime        1572 non-null   bool 
 12  Documentary  1572 non-null   bool 
 13  Drama        1572 non-null   bool 
 14  Fantasy      1572 non-null   bool 
 15  Film-Noir    1572 non-null   bool 
 16  Horror       1572 non-null   bool 
 17  IMAX         1572 non-null   bool 
 18  Musical      1572 non-null   bool 
 19  Mystery      157

## Create Model

   - ### Process


User enters movie Title

Key characteristics are extracted from the list and used as a fingerprint.

The title is removed from the original list.

The key characteristics of the title are matched with the data table.

The percentage is calculated.

The resulting data set is sorted and display top 10 highest matches.


In [53]:
# error catching will be put into final production. 
# this gets the point accross.
movie_title = "Step Brothers (2008)"  # replace with your movie title

movie_record = clean_model_data.loc[movie_title]

# Exclude the current movie from the DataFrame
movie_list = clean_model_data.drop(movie_title)

# Calculate the match score
match = np.dot(movie_list.values, movie_record.values) / np.sum(movie_record.values)

# Create a DataFrame for easier manipulation
match_df = pd.DataFrame(match, index=movie_list.index, columns=['Match Score'])

# Sort the DataFrame in descending order of match score
sorted_match_df = match_df.sort_values(by='Match Score', ascending=False)

sorted_match_df.head(10)

,Match Score
title,
(500) Days of Summer (2009),0.5
Oliver Twist (1948),0.5
"Outsiders, The (1983)",0.5
Out of the Past (1947),0.5
Out of Sight (1998),0.5
Out of Africa (1985),0.5
Our Hospitality (1923),0.5
"Other Sister, The (1999)",0.5
Othello (1995),0.5


## Write up

Data Extraction and Description
Our dataset is composed of four main components: movies, tags, ratings, and links.

Movies
The movies dataset is the core of our data. It is composed of three columns: id, title, and genre. The primary key is movieId. The genre column was split by the pipe character, and then pivoted along the columns to display boolean output. The granularity of this dataset is at the single movie level.

Tags
The tags dataset allows users to create hashtags on social media, providing brief characteristics of what the movie is about. The primary key is a combination of movieId and userId.

Ratings
The ratings dataset allows users to rate a movie from 1 to 5. We treated the rating value as an ordinal category by forcing it to be an integer. The primary key is a combination of movieId and userId.

Links
The links dataset serves as a bridge that can be used to tie in different database tables.


Data Integration and Aggregation
We integrated the datasets from the lowest level of granularity to the highest, rolling up the data to bring the granularity back to the movie level. This aggregated data would be used for the model. We chose to aggrigate and group the data on movie and use the mode of each boolean column to delive a result.

Model Implementation
The actual model follows the following process:


The user enters a movie title.<br>
That movie title is extracted formt the list and contains within itself a "fingerprint".
This fingerprint is saved and the removed from the original group.
The title's fingerprint is matched with the other titles fingerprints.
We calculated a match percentage based on the titles numer of 1's and the matching hits.
The resulting data set is sorted and the top 10 highest matches are displayed.
We leveraged the boolean values from the data preparation and basic arithmetic to derive similar titles based on the fingerprint of the title in question.
This approach allows us to provide users with movie recommendations that align closely with their input title.